# Aujourd'hui on roule sur les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html

# Imports des librairies de bases

On ajoutera celles qui manquent au fur et à mesure de nos besoins

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os, gc

# Définition de la seed pour le random

Très important pour qu'on voit les mêmes choses entre nos deux ordis

In [ ]:
RANDOM_SEED = 42;
np.random.seed(RANDOM_SEED)

# Définition des paramètres pour Matplot

Rien de bien intéréssant

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Set des variables globales

Attention, je n'utilise les variables globales pour la gestion des fichiers. Sinon, c'est mort

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "data_processed")

# Fonction pour load les libraires

En vrai, on a juste besoin de pd.read_csv, mais c'était pour faire joli

In [ ]:
def load_data(file,data_path=DATA_PATH, sep=','):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep)

# On load les jeux de données

Seulement le train (input_train) pour le moment. Et le "Y" des lignes dans le fichier avec le long nom.
Je ne sais pas pourquoi ils ont tout divisé, mais peût être pour filter les mecs qui ne savent pas fire des jointures.

In [ ]:
TX_data = load_data(file = "input_train.csv");
STA_data = load_data(file = "challenge_output_data_training_file_prediction_of_transaction_claims_status.csv", sep=';');


# Jointure entre les X et Y

In [ ]:
# Merge de TX_data et STA_data en utilisant les index comme cléf primaire
TX_data = pd.merge(TX_data, STA_data, left_index=True, right_index=True)

# On drop les ID qui sont crée dans le processus
TX_data.drop(["ID_y","ID_x"],inplace=True,axis=1)

# On applique tout ce qui peut l'être en numérique
TX_data=TX_data.apply(pd.to_numeric, errors='ignore')

del STA_data #Je supprime toujours les variables inutiles pour liberer ma RAM (qui est très faible)

# Fonction de mise en forme (attention, sportive)

Dans la BDD, j'ai vu pas mal de truc du genre "20<50", inutilisables. Donc, j'ai voulu splité pour faire une colonne avec le "20" (MIN), et une autre avec le "50" (MAX).
Donc j'ai fait cette fonction qui prend en entrée la BDD, une liste de noms de colones à traiter, et modifie la base de donnée directement.

In [ ]:
from rakuten import Splitter

list_col_split=["WARRANTIES_PRICE",
                'SELLER_SCORE_COUNT',
                'ITEM_PRICE',
                'PURCHASE_COUNT',
                'SHIPPING_PRICE']

# La classe "Splitter" prend en init (le constructeur) la liste des colonnes qu'elle va traiter
splitter_cell = Splitter(list_col=list_col_split)

# On applique la transformation sur le jeu de données
TX_data = splitter_cell.transform(TX_data)

del list_col_split, splitter_cell

In [ ]:
from rakuten import DateSplitter

# La classe "DateSplitter" prend en init (le constructeur) la liste des colonnes qu'elle va traiter
splitter_date = DateSplitter('BUYING_DATE')

# On applique la transformation sur le jeu de données
TX_data = splitter_date.transform(TX_data)

del splitter_date

# Feature Engineering

Là, faut être créatif, et aider l'algorithme à voir des choses, et se focaliser sur d'autres.

Je vais rarement drop des colonnes, car à ce moment là, je ne sais pas ce qui est utile ou pas

## A une garantie

In [ ]:
from rakuten import BoolMapper

# La classe "BoolMapper" prend en init (le constructeur) la liste des colonnes qu'elle va traiter
boolMap = BoolMapper(col='WARRANTIES_FLG')

# On applique la transformation sur le jeu de données
TX_data = boolMap.transform(TX_data)

del boolMap

## Taux de couverture de la garantie

In [ ]:
from rakuten import WarrantCov

# Ne prend rien en entrée, elle sait déjà quelle colonne elle va traiter (parce que c'est mal codé)
couverture = WarrantCov()

# On applique
TX_data = couverture.transform(TX_data)

del couverture

## Shipping Method

Regroupement des méthodes de shipping rares sous la même catégorie.

Objectif = Diminuer la complexité de la variable

In [ ]:
from rakuten import Dummyzator

list_rare_delivery_type = ['EXPRESS_DELIVERY',
                          'SO_RECOMMANDE',
                          'MONDIAL_RELAY',
                          'MONDIAL_RELAY_PREPAYE',
                          'SO_POINT_RELAIS',
                          'CHRONOPOST',
                          'PICKUP',
                          'Kiala']

# La classe "Dummyzator" prend en init (le constructeur):
# 1. La cible (colonne) pour remplacer les valeurs sur "target"
# 2. la liste des valeurs à remplacer sur "list_col"
# 3. La valeur à changer sur "new_value"

# Elle renvoit la même base de données, mais avec la cible dummyfiée (une colonne binaire par valeur de la cible)

dummyShip = Dummyzator(target = "SHIPPING_MODE", 
                     list_col = list_rare_delivery_type, 
                     new_value = "RARE_TYPE")

TX_data=dummyShip.transform(TX_data)

del list_rare_delivery_type, dummyShip

## Customer Premium

Regroupement des types de comptes payant sous la même catégorie.

Objectif = Diminuer la complexité de la variable

In [ ]:
list_rare_client_type= ['PLATINUM', 
                          'SILVER', 
                          'GOLD']

dummyPrice=Dummyzator(target = "PRICECLUB_STATUS",
                      list_col = list_rare_client_type,
                      new_value = "PREMIUM")

TX_data=dummyPrice.transform(TX_data)

del list_rare_client_type, dummyPrice

## Trajets longs

Flag pour signaler un envoi international

Objectif = souligner les courts trajets

In [ ]:
def f(row):
    long_haul_list = ["CHINA","HONG KONG", "UNITED STATES"]
    return 1 if row['SELLER_COUNTRY'] in long_haul_list else 0;

TX_data['LONG_HAUL'] = TX_data.apply(f, axis=1)

## Stats par Pays

To do : Détecter les pays à problèmes

In [ ]:
from rakuten import Stat_Adder

sell_country_stat = Stat_Adder(target = "SELLER_COUNTRY")

sell_country_stat.fit(TX_data)

TX_data = sell_country_stat.transform(TX_data)

del sell_country_stat

## Type de produit

To do : Détecter les produits fragiles

Objectif = souligner cet effet

In [ ]:
product_stat = Stat_Adder(target = "PRODUCT_TYPE")

product_stat.fit(TX_data)

TX_data = product_stat.transform(TX_data)

del product_stat

## Création d'un ID par client 

Objectif = créer une base de données client

In [ ]:
from rakuten import ID_Stat_Adder

id_Concat_buyer = ['REGISTRATION_DATE',"BUYER_DEPARTMENT","BUYER_BIRTHDAY_DATE"]

buyerID = ID_Stat_Adder(ID = 'BUYER_ID', 
                        concatenator = id_Concat_buyer)

buyerID.fit(TX_data)

TX_data = buyerID.transform(TX_data)

del id_Concat_buyer, buyerID

## Création d'un ID par vendeur

Objectif = créer une base de données vendeur

In [ ]:
id_Concat_seller = ['SELLER_COUNTRY',"SELLER_DEPARTMENT","SELLER_SCORE_AVERAGE"]

sellID = ID_Stat_Adder(ID = 'SELLER_ID', 
                        concatenator = id_Concat_seller)

sellID.fit(TX_data)

TX_data = sellID.transform(TX_data)

del id_Concat_seller, sellID

## Fusion des bases de données

Objectif = Yolo

In [ ]:
list_to_drop=["BUYER_ID","SELLER_ID", "0_x", "0_y"]

In [ ]:
TX_data.drop(list_to_drop,inplace=True,axis=1)

del list_to_drop

In [ ]:
gc.collect()

# On essaie d'ajouter des composantes de Neighbors, PCA, LDA, etc..

In [ ]:
def datapreprocess_LDA(data):
    data=data.apply(pd.to_numeric, errors='ignore')
    
    # Y and X
    try: 
        Y=data["CLAIM_TYPE"]
        X=data.drop("CLAIM_TYPE", axis=1,inplace=False)
    except:
        Y=0
        X=data;
        pass;
    # Exclude Objets
    X=X.select_dtypes(exclude=['object'])
    
    # Work on fare
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    X=pd.DataFrame(imp.fit_transform(X),columns=X.columns.values)
 
    return X, Y

In [ ]:
X,Y=datapreprocess_LDA(TX_data)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_r = pd.DataFrame(pca.fit(X).transform(X), columns=['PCA_1', 'PCA_2'])
TX_data['PCA_1']=X_r['PCA_1']
TX_data['PCA_2']=X_r['PCA_2']

del datapreprocess_LDA, pca, X_r, lda, X_r2
gc.collect()

# Et on écrit la BDD sur un joli CSV

In [ ]:
filename = DATA_PROCESSED+"/working_data.csv"

In [ ]:
os.remove(filename)

In [ ]:
TX_data.to_csv(path_or_buf=filename, index=False)

# A partir de là, je fais juste un test bidon pour voir si les features apportent quelque chose

## Les vrais modèles sont dans le notebook "Models"

## On coupe

Objectif = Créer un jeu de données train et test

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(TX_data, 
                                       test_size=0.3, 
                                       random_state=RANDOM_SEED, 
                                       stratify=TX_data["CLAIM_TYPE"]
                                      )
del TX_data;

## Preprocess

Objectif = Rendre joli tout ça

In [ ]:
def datapreprocess(data):
    data=data.apply(pd.to_numeric, errors='ignore')
    
    # Y and X
    try :
        Y=data["CLAIM_TYPE"]
        X=data.drop("CLAIM_TYPE", axis=1,inplace=False)
    except:
        Y=0
        X=data
    # Exclude Objets
    X=X.select_dtypes(exclude=['object'])
    
    # Work on fare
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    X=pd.DataFrame(imp.fit_transform(X),columns=X.columns.values)
 
    return X, Y

In [ ]:
X_train,Y_train=datapreprocess(train_set)
X_test, Y_test=datapreprocess(test_set)

del train_set, test_set;
gc.collect()

# MODEL!

## Metriques

D'abord, notre métrique à nous

In [ ]:
from rakuten import multiclass_roc_auc_score

#### Notre Random Forest tout simple

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rnd_clf = RandomForestClassifier(n_jobs=-1, 
                                 random_state=RANDOM_SEED)

In [ ]:
rnd_clf.fit(X_train, Y_train);
y_pred_rf = rnd_clf.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_rf)

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import SGDClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(max_iter=5, random_state=42))

In [ ]:
ovo_clf.fit(X_train, Y_train)
y_pred_ovo=ovo_clf.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_ovo)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)
    
conf_mx = confusion_matrix(Y_test, y_pred_ovo)
plot_confusion_matrix(conf_mx)

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)

In [ ]:
from rakuten import rakuten_ROC_CV

rakuten_ROC_CV(rnd_clf, X_train, Y_train, _n_splits=10, _random_state=RANDOM_SEED)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, 
                             random_state=RANDOM_SEED)

In [ ]:
sfm_clf = SelectFromModel(clf)
del clf

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb_clf=GradientBoostingClassifier(random_state=RANDOM_SEED)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [ ]:
clf = Pipeline([
    #('scaler', StandardScaler()), 
    #('norm', Normalizer()), 
    #('threshold', sel_vt), 
    #('SelectKBest', Chi_select), 
    #('reduce_dim', PCA()),
    #('feature_union', combined),
    #('feature_selection', sfm_clf),
    ('classification', gss)
])


clf.fit(X_train, Y_train);
y_pred_rf = clf.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_rf)

In [ ]:
conf_mx = confusion_matrix(Y_test, y_pred_rf)
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
print(model)

In [ ]:
y_pred_xgb = model.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_xgb)

In [ ]:
conf_mx = confusion_matrix(Y_test, y_pred_xgb)
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
plot_confusion_matrix(norm_conf_mx)

In [ ]:
len(Y_train.unique())

In [ ]:
from sklearn.decomposition import PCA

target_names = Y_train.unique()

pca = PCA(n_components=2)
X_r = pca.fit(X_train).transform(X_train)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))


In [ ]:
toto=Y_train.reset_index()
toto.drop("index", inplace=True, axis=1)
X_r=pd.DataFrame(X_r, columns=["one","two"])
X_r['result']=toto

In [ ]:
import seaborn as sns
g = sns.FacetGrid(X_r, hue="result", size=7)
g.map(plt.scatter, "one", "two", s=50, alpha=.7, linewidth=.5, edgecolor="white")
g.add_legend();

In [ ]:
import seaborn as sns
sns.pairplot(X_train);